# Borda Count... von Count

At a glance Borda count is an extension of plurality voting. At it's core this method is point-based ranking, a voter's prefered candidate gets the most points. The winner is the candidate with the most points in the election.

### How does it work?

Suppose there are $n$ candidates in an election and a voter ranks these candidates. Borda count assigns weights to each  candidate based on their placement in the voter's ranking. These weights are usually positive numbers, and follow some linear scale.

The common weights for an $n$ candidate election are 

$$n, n-1, \dots, 2, 1$$. 

Where the candidate ranked first by a voter gets $n$ points and the one ranked last gets 1 point.

Consider the following example, 

|      | Pizza| Tacos| Burgers | Curry | BBQ |
|------|------|------|------|------|------|
|Alice |   4  |   3  |   5  |  2   |   1  |

Alice is really in the mood for burgers so she ranked it the highest. If we use the common weights, we don't do anything to this ranking as $burgers$ gets 5 points already.

## Let's Complicate Things:


In [9]:
import numpy as np
import pandas as pd

f = open('test_data.txt')
candidates = f.readline().split()
f.close    
        
data = np.loadtxt('test_data.txt', skiprows = 1)
matrix = pd.DataFrame(data)
print(data)
#print(candidates)

[[5. 4. 3. 2. 1.]
 [5. 1. 4. 3. 2.]
 [2. 5. 4. 1. 3.]
 [5. 1. 3. 2. 4.]
 [2. 4. 3. 5. 1.]
 [1. 2. 4. 5. 3.]]


When we run the code above, this is how our data looks like.


|        |  A  |  B  |  C  |  D  |  E  |
|--------|-----|-----|-----|-----|-----|
|Voter 1 |  5  |  4  |  3  |  2  |  1  |
|Voter 2 |  5  |  1  |  4  |  3  |  2  |
|Voter 3 |  2  |  5  |  4  |  1  |  3  |
|Voter 4 |  5  |  1  |  3  |  2  |  4  |
|Voter 5 |  2  |  4  |  3  |  5  |  1  |
|Voter 6 |  1  |  2  |  4  |  5  |  3  |
|  $\textbf{Total:}$|  20 |  17 |  21 |  18 |  14 |


The code below gives us the total of points given to candidates in the election.

In [10]:
results = data.sum(axis = 0)
print(results)

[20. 17. 21. 18. 14.]


# Notice Something?

If we look at our results in plurality voting we had the following results, 


$$ \textbf{Plurality} $$

|        |  A  |  B  |  C  |  D  |  E  |
|--------|-----|-----|-----|-----|-----|
|Results |  3  |  1  |  0  |  2  |  0  |

If we rank candidates from first to fifth place and compare them with Borda count we get, 

$$ \textbf{ Plurality v.s Borda Count} $$

|            |      A     |      B     |     C     |      D     |      E     |
|------------|------------|------------|-----------|------------|------------|
|Plurality   |  $1^{st}$  |  $3^{rd}$  |  $4^{th}$ |  $2^{nd}$  |  $4^{th}$  |
|Borda Count |  $2^{nd}$  |  $4^{th}$  |  $1^{st}$ |  $3^{rd}$  |  $5^{th}$  |

Notice that eventhough Candidate A wins in plurality. The ranks of other candidates are changed. So there is a possiblity that a candidate can win even if they are the preferred choice of various voters.

## I Don't Like  Weights

If other weights are prefered we use the following code to find and count each candidates instance in each place. The loop below will output the following table.

|        |  A  |  B  |  C  |  D  |  E  |
|--------|-----|-----|-----|-----|-----|
| First  |  3  |  0  |  0  |  2  |  1  |
| Second |  1  |  2  |  0  |  1  |  2  |
| Third  |  0  |  3  |  3  |  0  |  0  |
| Fourth |  2  |  0  |  1  |  2  |  1  |
| Fifth  |  0  |  1  |  2  |  1  |  2  |

Notice that the "First" row shows the results of Plurality. From here we can assign weights and determine the winner.

### How does it work?

- We find the shape of the data matrix.
- Create a matrix ```ranked_counts``` that will count the times a candidate appears in each rank.
- Use a nested for loop that does the following:

    - Compute ``` row_max ```.
    - For $candidate_i$ find the location of the ``` row_max ```.
    - Increase the times $candidate_i$ appears in $rank_j$.
    - Replace the ``` row max ``` with 0.
    - Repeat...
    

In [11]:
row, col = np.shape(matrix - 1)
   
ranked_counts = np.zeros((row - 1,col))
print(matrix.idxmax(axis = 1))  
while data.any() != np.zeros((row, col)).any():
    for i in range(row):
        for j in range(col):
            
            #find the row max.
            row_max = matrix.idxmax(axis = 1)
             
            #find the location of the row max for candidate_i.
            location = row_max[i]
            
            #Increase the instance of candidate_i in rank_j
            ranked_counts[location,j] += 1
            
            #replace the location of the row max with 0.
            data[i, location] = 0
            
            
print(ranked_counts)

0    0
1    0
2    1
3    0
4    3
5    3
dtype: int64
[[3. 0. 0. 2. 1.]
 [1. 2. 0. 1. 2.]
 [0. 3. 3. 0. 0.]
 [2. 0. 1. 2. 1.]
 [0. 1. 2. 1. 2.]]


# Do You Even Borda?

Let's assign some weights, all we need is an $n$-length array. This array will be our $n \times 1$ vector of weights that we multiply the position matrix by. Once this happens we just sum up our row and **BAM!** we have our points. Let's say for this example, we want to give more points to ranks 1 and 2, and significantly less points to the rest of the candidates.

$$ weights = (15, 10, 3, 2, 1) $$

Since we already have the ``` ranked_counts ``` matrix, we just need to multiply the matrix with the weights vector. By the structure of the ``` ranked_counts ``` matrix multiplication, out Borda points are given by the row sum.


In [4]:
#Assign weights.
weights = [15, 10, 3, 2 ,1]

#Multiply the weights and ranked_counts.
borda_matrix = weights * ranked_counts

#Get the row sum to find the points of each candidate.
borda_points = borda_matrix.sum(axis = 1)


print('Here are the election results:')
for i in range(len(borda_points)):
    print('{}: '.format(candidates[i]), str(borda_points[i]))

Here are the election results:
A:  50.0
B:  39.0
C:  39.0
D:  38.0
E:  20.0


With these weights our results are as follow, 

|        |    A    |    B    |    C    |    D    |   E    |
|--------|---------|---------|---------|---------|--------|
|Results |    50   |   39    |   39    |   38    |   20   |
|Ranking | $1^{st}$| $2^{nd}$| $2^{nd}$| $3^{rd}$|$4^{th}$|

# Issues

Just as in plurality, we again run into issues with ties. While  our example had a tie for $2^{nd}$ place, we are interested in first place only so we can "ignore" it. In case there's a tie for first, again we would need a run-off election, which again, is not possible with our data.

